Codigo para analise de empregados no setor de serviço

In [1]:
#bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sidrapy import get_table
import sidrapy
import statsmodels.api as sm
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller



In [ ]:

# Definindo a URL da API
url = "https://apisidra.ibge.gov.br/values/t/6449/n1/all/v/2585/p/all/c12762/117756"

# Requisição GET para obter os dados
response = requests.get(url)
data = response.json()

# Convertendo os dados para um DataFrame
df = pd.DataFrame(data)

# Exibindo as primeiras linhas do DataFrame para verificar a estrutura dos dados
print(df.head())

# Selecionando apenas as colunas 'D3N' (ano) e 'V' (valor)
df = df[['D3N', 'V']]

# Renomeando as colunas para 'ano' e 'valor'
df.columns = ['ano', 'valor']

# Filtrando as linhas onde o valor da coluna 'ano' não é "Ano"
df = df[df['ano'] != 'Ano']

# Convertendo a coluna 'ano' para o formato datetime
df['ano'] = pd.to_datetime(df['ano'], format='%Y')

# Convertendo a coluna 'valor' para numérico (forçando erro se não for numérico)
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

# Removendo linhas onde o valor não pôde ser convertido para numérico
df = df.dropna(subset=['valor'])

# Ordenando os dados por ano
df = df.sort_values('ano')

# Exibindo o gráfico da evolução dos valores ao longo dos anos
df.set_index('ano')['valor'].plot()
plt.xlabel('Ano')
plt.ylabel('Valor')
plt.title('Evolução dos valores ao longo dos anos')
plt.show()



In [ ]:
# Definindo o índice como o ano para análise de séries temporais
df.set_index('ano', inplace=True)

# 1. Análise de Estacionariedade (Teste de Dickey-Fuller)
result = adfuller(df['valor'])
print(f'p-value: {result[1]}')

# Se p-value for maior que 0.05, a série não é estacionária e precisa ser diferenciada
if result[1] > 0.05:
    print("A série não é estacionária, aplicando diferenciação.")
    df['valor_diff'] = df['valor'].diff().dropna()
else:
    print("A série é estacionária.")

# 2. Ajuste automático do modelo ARIMA
# Ajustando manualmente o modelo ARIMA (porque o auto_arima do pmdarima não é usado aqui)
# Você pode tentar ajustar a ordem conforme necessário (p, d, q)
model = ARIMA(df['valor_diff'].dropna(), order=(1, 1, 1))  # Exemplo de (p, d, q)
model_fit = model.fit()

# Exibindo o resumo do modelo
print(model_fit.summary())

# 3. Ajustando o modelo ARIMA com os parâmetros encontrados
# Aqui, o modelo já foi ajustado, então vamos usar esse ajuste para previsões

# 4. Fazendo previsões para os próximos 5 anos
forecast_steps = 5  # Número de passos para previsão
forecast = model_fit.forecast(steps=forecast_steps)

# Gerando o índice para as previsões
forecast_index = pd.date_range(start=df.index[-1], periods=forecast_steps + 1, freq='A')[1:]

# Plotando as previsões
plt.figure(figsize=(10,6))
plt.plot(df.index, df['valor'], label='Dados históricos')
plt.plot(forecast_index, forecast, label='Previsões', color='red')
plt.xlabel('Ano')
plt.ylabel('Valor')
plt.title('Previsões ARIMA')
plt.legend()
plt.show()

# 5. Calculando o erro do modelo (exemplo com RMSE)
# Calculando o erro entre os valores previstos e os reais
predicted_values = model_fit.predict(start=0, end=len(df) - 1)
rmse = np.sqrt(mean_squared_error(df['valor'].dropna(), predicted_values))
print(f"Erro Quadrático Médio (RMSE): {rmse}")


In [ ]:
# Verificando estacionariedade novamente após a diferenciação
adf_test = adfuller(df['valor_diff'].dropna())
print(f'p-value após a diferenciação: {adf_test[1]}')


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Ajustando o modelo SARIMA com os melhores parâmetros encontrados
sarima_opt_model = SARIMAX(df['valor'], 
                           order=(1, 1, 1),  # Ajuste esses valores com base no seu melhor modelo
                           seasonal_order=(1, 1, 1, 12))  # '12' é exemplo; ajuste para a periodicidade dos seus dados
sarima_opt_fit = sarima_opt_model.fit(disp=False)

# Previsões com o modelo ajustado
forecast_steps = 5
forecast = sarima_opt_fit.get_forecast(steps=forecast_steps)
forecast_conf_int = forecast.conf_int()

# Plotando as previsões
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['valor'], label='Dados históricos')
plt.plot(forecast.predicted_mean.index, forecast.predicted_mean, label='Previsões', color='red',linestyle='--')
plt.fill_between(forecast_conf_int.index, 
                 forecast_conf_int.iloc[:, 0], 
                 forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.xlabel('Ano')
plt.ylabel('Valor')
plt.title('Previsões com SARIMA')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(df['valor'].dropna(), sarima_opt_fit.predict(start=0, end=len(df)-1))
print(f"Erro Absoluto Médio (MAE): {mae}")
